In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


In [2]:
# read the data provided
train=pd.read_csv("C:/Users/new/Desktop/train_V2.csv")
test=pd.read_csv("C:/Users/new/Desktop/test_V2.csv")
sample_submission=pd.read_csv("C:/Users/new/Desktop/sample_submission_V2.csv")

In [3]:
# what the train data looks like
train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [4]:
# basically the same thing
test.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
0,9329eb41e215eb,676b23c24e70d6,45b576ab7daa7f,0,0,51.46,0,0,0,73,...,1500,0,0.0,0,0.0,0,0,588.0,1,0
1,639bd0dcd7bda8,430933124148dd,42a9a0b906c928,0,4,179.10,0,0,2,11,...,1503,2,4669.0,0,0.0,0,0,2017.0,6,0
2,63d5c8ef8dfe91,0b45f5db20ba99,87e7e4477a048e,1,0,23.40,0,0,4,49,...,1565,0,0.0,0,0.0,0,0,787.8,4,0
3,cf5b81422591d1,b7497dbdc77f4a,1b9a94f1af67f1,0,0,65.52,0,0,0,54,...,1465,0,0.0,0,0.0,0,0,1812.0,3,0
4,ee6a295187ba21,6604ce20a1d230,40754a93016066,0,4,330.20,1,2,1,7,...,1480,1,0.0,0,0.0,0,0,2963.0,4,0


In [5]:
# to find out the nan and columns that are not usable.
train.isnull().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       1
dtype: int64

In [6]:
#since we are trying to predict the winPlacePer, we are droping it.
print('train shape before dropping null=',train.shape)
train=train.dropna()
print('train shape after dropping null=',train.shape)

train shape before dropping null= (4446966, 29)
train shape after dropping null= (4446965, 29)


In [7]:
y=train['winPlacePerc']
train=train.drop(columns=['winPlacePerc','Id','groupId','matchId'],axis=1)
test=test.drop(columns=['Id','groupId','matchId'],axis=1)

In [8]:
# this is the train dummies function that I was going to go over in the class but we uploaded the old ppt
# what this basically means is that setting different variables with different values to 0/1. The value itself is not important, but its something we use to distinguish the difference between the data. However, in analyzing we would have to make them 0 and 1 to be meaningful.
train=pd.get_dummies(train)
test=pd.get_dummies(test)
print('train shape:',train.shape,'test shape:',test.shape)
# as we see over here, there are 11 more values that had been labeled as 1 which means meaningful.

train shape: (4446965, 40) test shape: (1934174, 40)


In [9]:
from sklearn.model_selection import train_test_split


In [10]:
x_train,x_test,y_train,y_test=train_test_split(train,y,test_size=0.2)
#train test split and set the testing data to 0.2

In [13]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
#importing catboost, which stands for catagorical boosting

In [14]:
#xgb=XGBRegressor(max_depth=18,ganmma=0.5,n_jobs=20)
#lgb=LGBMRegressor()
cb=CatBoostRegressor(iterations=2500,depth=10)
#rf=RandomForestRegressor()

In [15]:
#xgb.fit(x_train,y_train)
cb.fit(x_train,y_train)
#rf.fit(x_train,y_train)

0:	learn: 0.5477805	total: 3.65s	remaining: 2h 32m 1s
1:	learn: 0.5321601	total: 7.04s	remaining: 2h 26m 31s
2:	learn: 0.5170046	total: 10.4s	remaining: 2h 24m 25s
3:	learn: 0.5023294	total: 13.7s	remaining: 2h 22m 37s
4:	learn: 0.4881274	total: 17.1s	remaining: 2h 21m 58s
5:	learn: 0.4743628	total: 22.3s	remaining: 2h 34m 38s
6:	learn: 0.4610423	total: 25.8s	remaining: 2h 33m 5s
7:	learn: 0.4481539	total: 29.5s	remaining: 2h 33m 23s
8:	learn: 0.4356059	total: 33.1s	remaining: 2h 32m 35s
9:	learn: 0.4234738	total: 36.8s	remaining: 2h 32m 45s
10:	learn: 0.4117316	total: 40.6s	remaining: 2h 33m 5s
11:	learn: 0.4003371	total: 44.4s	remaining: 2h 33m 16s
12:	learn: 0.3893163	total: 48.2s	remaining: 2h 33m 42s
13:	learn: 0.3786239	total: 52s	remaining: 2h 33m 59s
14:	learn: 0.3683131	total: 55.6s	remaining: 2h 33m 38s
15:	learn: 0.3583270	total: 59.3s	remaining: 2h 33m 33s
16:	learn: 0.3486356	total: 1m 3s	remaining: 2h 33m 31s
17:	learn: 0.3392804	total: 1m 6s	remaining: 2h 33m 14s
18:	lea

145:	learn: 0.0900642	total: 9m 10s	remaining: 2h 27m 59s
146:	learn: 0.0900006	total: 9m 14s	remaining: 2h 27m 56s
147:	learn: 0.0899108	total: 9m 18s	remaining: 2h 27m 53s
148:	learn: 0.0898285	total: 9m 22s	remaining: 2h 27m 50s
149:	learn: 0.0897646	total: 9m 26s	remaining: 2h 27m 47s
150:	learn: 0.0896568	total: 9m 29s	remaining: 2h 27m 45s
151:	learn: 0.0895830	total: 9m 33s	remaining: 2h 27m 44s
152:	learn: 0.0895086	total: 9m 37s	remaining: 2h 27m 41s
153:	learn: 0.0894304	total: 9m 41s	remaining: 2h 27m 38s
154:	learn: 0.0893517	total: 9m 45s	remaining: 2h 27m 35s
155:	learn: 0.0892774	total: 9m 49s	remaining: 2h 27m 33s
156:	learn: 0.0892080	total: 9m 53s	remaining: 2h 27m 30s
157:	learn: 0.0891253	total: 9m 56s	remaining: 2h 27m 27s
158:	learn: 0.0890570	total: 10m	remaining: 2h 27m 24s
159:	learn: 0.0889899	total: 10m 4s	remaining: 2h 27m 21s
160:	learn: 0.0889380	total: 10m 8s	remaining: 2h 27m 18s
161:	learn: 0.0888608	total: 10m 12s	remaining: 2h 27m 15s
162:	learn: 0.08

286:	learn: 0.0839427	total: 18m 20s	remaining: 2h 21m 25s
287:	learn: 0.0839208	total: 18m 24s	remaining: 2h 21m 21s
288:	learn: 0.0839013	total: 18m 28s	remaining: 2h 21m 18s
289:	learn: 0.0838734	total: 18m 32s	remaining: 2h 21m 15s
290:	learn: 0.0838551	total: 18m 36s	remaining: 2h 21m 12s
291:	learn: 0.0838306	total: 18m 39s	remaining: 2h 21m 8s
292:	learn: 0.0838038	total: 18m 43s	remaining: 2h 21m 4s
293:	learn: 0.0837845	total: 18m 47s	remaining: 2h 21m 1s
294:	learn: 0.0837544	total: 18m 52s	remaining: 2h 21m 7s
295:	learn: 0.0837322	total: 18m 56s	remaining: 2h 21m 3s
296:	learn: 0.0837092	total: 19m	remaining: 2h 21m
297:	learn: 0.0836781	total: 19m 4s	remaining: 2h 20m 57s
298:	learn: 0.0836654	total: 19m 8s	remaining: 2h 20m 53s
299:	learn: 0.0836412	total: 19m 12s	remaining: 2h 20m 49s
300:	learn: 0.0836190	total: 19m 16s	remaining: 2h 20m 45s
301:	learn: 0.0835920	total: 19m 19s	remaining: 2h 20m 42s
302:	learn: 0.0835741	total: 19m 23s	remaining: 2h 20m 38s
303:	learn: 

427:	learn: 0.0817339	total: 27m 30s	remaining: 2h 13m 9s
428:	learn: 0.0817241	total: 27m 34s	remaining: 2h 13m 5s
429:	learn: 0.0817125	total: 27m 38s	remaining: 2h 13m 1s
430:	learn: 0.0816962	total: 27m 41s	remaining: 2h 12m 58s
431:	learn: 0.0816893	total: 27m 45s	remaining: 2h 12m 54s
432:	learn: 0.0816751	total: 27m 50s	remaining: 2h 12m 54s
433:	learn: 0.0816650	total: 27m 54s	remaining: 2h 12m 50s
434:	learn: 0.0816580	total: 27m 58s	remaining: 2h 12m 47s
435:	learn: 0.0816441	total: 28m 2s	remaining: 2h 12m 44s
436:	learn: 0.0816391	total: 28m 6s	remaining: 2h 12m 40s
437:	learn: 0.0816294	total: 28m 10s	remaining: 2h 12m 37s
438:	learn: 0.0816184	total: 28m 14s	remaining: 2h 12m 33s
439:	learn: 0.0816104	total: 28m 18s	remaining: 2h 12m 30s
440:	learn: 0.0815963	total: 28m 22s	remaining: 2h 12m 26s
441:	learn: 0.0815862	total: 28m 25s	remaining: 2h 12m 22s
442:	learn: 0.0815770	total: 28m 29s	remaining: 2h 12m 19s
443:	learn: 0.0815687	total: 28m 33s	remaining: 2h 12m 15s
44

569:	learn: 0.0806036	total: 37m 3s	remaining: 2h 5m 28s
570:	learn: 0.0805992	total: 37m 7s	remaining: 2h 5m 25s
571:	learn: 0.0805926	total: 37m 11s	remaining: 2h 5m 22s
572:	learn: 0.0805874	total: 37m 15s	remaining: 2h 5m 18s
573:	learn: 0.0805804	total: 37m 19s	remaining: 2h 5m 14s
574:	learn: 0.0805731	total: 37m 23s	remaining: 2h 5m 10s
575:	learn: 0.0805689	total: 37m 27s	remaining: 2h 5m 6s
576:	learn: 0.0805636	total: 37m 31s	remaining: 2h 5m 3s
577:	learn: 0.0805562	total: 37m 35s	remaining: 2h 4m 59s
578:	learn: 0.0805511	total: 37m 39s	remaining: 2h 4m 55s
579:	learn: 0.0805462	total: 37m 42s	remaining: 2h 4m 51s
580:	learn: 0.0805410	total: 37m 47s	remaining: 2h 4m 48s
581:	learn: 0.0805337	total: 37m 51s	remaining: 2h 4m 47s
582:	learn: 0.0805265	total: 37m 55s	remaining: 2h 4m 43s
583:	learn: 0.0805196	total: 38m	remaining: 2h 4m 40s
584:	learn: 0.0805127	total: 38m 4s	remaining: 2h 4m 37s
585:	learn: 0.0805091	total: 38m 8s	remaining: 2h 4m 33s
586:	learn: 0.0805011	to

...

1933:	learn: 0.0777978	total: 2h 6m 41s	remaining: 37m 4s
1934:	learn: 0.0777969	total: 2h 6m 45s	remaining: 37m
1935:	learn: 0.0777961	total: 2h 6m 48s	remaining: 36m 56s
1936:	learn: 0.0777940	total: 2h 6m 52s	remaining: 36m 52s
1937:	learn: 0.0777925	total: 2h 6m 56s	remaining: 36m 48s
1938:	learn: 0.0777913	total: 2h 7m	remaining: 36m 44s
1939:	learn: 0.0777908	total: 2h 7m 4s	remaining: 36m 40s
1940:	learn: 0.0777896	total: 2h 7m 8s	remaining: 36m 36s
1941:	learn: 0.0777888	total: 2h 7m 12s	remaining: 36m 32s
1942:	learn: 0.0777878	total: 2h 7m 16s	remaining: 36m 29s
1943:	learn: 0.0777861	total: 2h 7m 19s	remaining: 36m 25s
1944:	learn: 0.0777849	total: 2h 7m 23s	remaining: 36m 21s
1945:	learn: 0.0777840	total: 2h 7m 27s	remaining: 36m 17s
1946:	learn: 0.0777829	total: 2h 7m 31s	remaining: 36m 13s
1947:	learn: 0.0777821	total: 2h 7m 35s	remaining: 36m 9s
1948:	learn: 0.0777809	total: 2h 7m 39s	remaining: 36m 5s
1949:	learn: 0.0777802	total: 2h 7m 43s	remaining: 36m 1s
1950:	learn

2072:	learn: 0.0776660	total: 2h 15m 39s	remaining: 27m 56s
2073:	learn: 0.0776651	total: 2h 15m 43s	remaining: 27m 52s
2074:	learn: 0.0776648	total: 2h 15m 47s	remaining: 27m 48s
2075:	learn: 0.0776640	total: 2h 15m 51s	remaining: 27m 44s
2076:	learn: 0.0776629	total: 2h 15m 54s	remaining: 27m 40s
2077:	learn: 0.0776621	total: 2h 15m 58s	remaining: 27m 36s
2078:	learn: 0.0776611	total: 2h 16m 2s	remaining: 27m 32s
2079:	learn: 0.0776600	total: 2h 16m 6s	remaining: 27m 29s
2080:	learn: 0.0776597	total: 2h 16m 10s	remaining: 27m 25s
2081:	learn: 0.0776587	total: 2h 16m 14s	remaining: 27m 21s
2082:	learn: 0.0776580	total: 2h 16m 18s	remaining: 27m 17s
2083:	learn: 0.0776571	total: 2h 16m 21s	remaining: 27m 13s
2084:	learn: 0.0776564	total: 2h 16m 25s	remaining: 27m 9s
2085:	learn: 0.0776556	total: 2h 16m 29s	remaining: 27m 5s
2086:	learn: 0.0776545	total: 2h 16m 33s	remaining: 27m 1s
2087:	learn: 0.0776535	total: 2h 16m 37s	remaining: 26m 57s
2088:	learn: 0.0776528	total: 2h 16m 41s	rema

2210:	learn: 0.0775459	total: 2h 24m 38s	remaining: 18m 54s
2211:	learn: 0.0775449	total: 2h 24m 42s	remaining: 18m 50s
2212:	learn: 0.0775440	total: 2h 24m 46s	remaining: 18m 46s
2213:	learn: 0.0775431	total: 2h 24m 50s	remaining: 18m 42s
2214:	learn: 0.0775422	total: 2h 24m 54s	remaining: 18m 38s
2215:	learn: 0.0775412	total: 2h 24m 59s	remaining: 18m 34s
2216:	learn: 0.0775406	total: 2h 25m 3s	remaining: 18m 30s
2217:	learn: 0.0775401	total: 2h 25m 7s	remaining: 18m 27s
2218:	learn: 0.0775394	total: 2h 25m 11s	remaining: 18m 23s
2219:	learn: 0.0775389	total: 2h 25m 15s	remaining: 18m 19s
2220:	learn: 0.0775381	total: 2h 25m 18s	remaining: 18m 15s
2221:	learn: 0.0775375	total: 2h 25m 22s	remaining: 18m 11s
2222:	learn: 0.0775371	total: 2h 25m 26s	remaining: 18m 7s
2223:	learn: 0.0775363	total: 2h 25m 30s	remaining: 18m 3s
2224:	learn: 0.0775357	total: 2h 25m 34s	remaining: 17m 59s
2225:	learn: 0.0775345	total: 2h 25m 38s	remaining: 17m 55s
2226:	learn: 0.0775337	total: 2h 25m 42s	rem

2348:	learn: 0.0774329	total: 2h 33m 46s	remaining: 9m 53s
2349:	learn: 0.0774323	total: 2h 33m 50s	remaining: 9m 49s
2350:	learn: 0.0774311	total: 2h 33m 54s	remaining: 9m 45s
2351:	learn: 0.0774303	total: 2h 33m 58s	remaining: 9m 41s
2352:	learn: 0.0774289	total: 2h 34m 2s	remaining: 9m 37s
2353:	learn: 0.0774278	total: 2h 34m 6s	remaining: 9m 33s
2354:	learn: 0.0774269	total: 2h 34m 10s	remaining: 9m 29s
2355:	learn: 0.0774261	total: 2h 34m 14s	remaining: 9m 25s
2356:	learn: 0.0774255	total: 2h 34m 18s	remaining: 9m 21s
2357:	learn: 0.0774247	total: 2h 34m 22s	remaining: 9m 17s
2358:	learn: 0.0774242	total: 2h 34m 25s	remaining: 9m 13s
2359:	learn: 0.0774235	total: 2h 34m 29s	remaining: 9m 9s
2360:	learn: 0.0774227	total: 2h 34m 33s	remaining: 9m 5s
2361:	learn: 0.0774211	total: 2h 34m 38s	remaining: 9m 2s
2362:	learn: 0.0774200	total: 2h 34m 42s	remaining: 8m 58s
2363:	learn: 0.0774193	total: 2h 34m 46s	remaining: 8m 54s
2364:	learn: 0.0774186	total: 2h 34m 49s	remaining: 8m 50s
23

2488:	learn: 0.0773228	total: 2h 43m 5s	remaining: 43.2s
2489:	learn: 0.0773219	total: 2h 43m 9s	remaining: 39.3s
2490:	learn: 0.0773214	total: 2h 43m 13s	remaining: 35.4s
2491:	learn: 0.0773205	total: 2h 43m 17s	remaining: 31.5s
2492:	learn: 0.0773197	total: 2h 43m 21s	remaining: 27.5s
2493:	learn: 0.0773186	total: 2h 43m 25s	remaining: 23.6s
2494:	learn: 0.0773178	total: 2h 43m 29s	remaining: 19.7s
2495:	learn: 0.0773173	total: 2h 43m 33s	remaining: 15.7s
2496:	learn: 0.0773159	total: 2h 43m 37s	remaining: 11.8s
2497:	learn: 0.0773153	total: 2h 43m 41s	remaining: 7.86s
2498:	learn: 0.0773146	total: 2h 43m 45s	remaining: 3.93s
2499:	learn: 0.0773132	total: 2h 43m 49s	remaining: 0us


In [16]:
y_pred=cb.predict(x_test)

In [17]:
#y_pred=y_pred/(y_pred.max()-y_pred.min())

In [18]:
from sklearn.metrics import r2_score

In [19]:
print('r2 score is:',r2_score(y_test,y_pred))

r2 score is: 0.9347845181197336


In [20]:
y_pred_final=cb.predict(test)

In [21]:
sample_submission['winPlacePerc']=y_pred_final

In [22]:
sample_submission.head(2)

,Id,winPlacePerc
0,9329eb41e215eb,0.250987
1,639bd0dcd7bda8,0.953512


In [23]:
sample_submission.to_csv('submission_pubg.csv',index=False)